In [1]:
import numpy as np
import galois
import random
bits = 128

from collections import defaultdict
from collections import Counter
from collections import namedtuple

In [2]:
def keygen(bits):
    """Generates keys with `bits`-bits of security. Returns a pair: (secret key, public key)."""
    def invmod(x, m):
        gcd, s, t = galois.egcd(x, m)
        assert gcd == 1
        return s

    p = galois.random_prime(int(bits/2))
    q = galois.random_prime(int(bits/2))

    n = p*q
    g = n+1
    lamb = (p-1) * (q-1)
    mu = invmod(lamb, n)
    
    sk = (lamb, mu)
    pk = (n, g)
    return sk, pk

def encrypt(m, pk):
    """Encrypts the message `m` with public key `pk`."""
    n, g = pk
    n_sq = n**2
    r = random.randint(1, n)
    c = (pow(g, m, n_sq) * pow(r, n, n_sq)) % n_sq
    return c

def decrypt(c, sk, pk):
    """Decrypts the ciphertext `c` using secret key `sk` and public key `pk`."""
    lamb, mu = sk
    n, g = pk
    n_sq = n**2
    L_result = (pow(c, lamb, n_sq) - 1)//n
    return (L_result * mu) % n

def e_add(c1, c2, pk):
    """Add one encrypted integer to another"""
    n, g = pk

    return c1 * c2 % n**2

In [3]:
class GenderPayGapSurveyParticipant:
    def submit_salary(self, salary, gender, server):
        """Submits an encrypted survey response to the server"""
        pk = server.get_public_key()
        gender_dict = {"Male": 1 , "Female": 0}
        temp_sal = [0, 0]
        temp_gen = [0, 0]
        gender_dict_ind = gender_dict[gender]
        temp_sal[gender_dict_ind] = salary
        temp_gen[gender_dict_ind] = 1
        enc_gen = [encrypt(gen, pk) for gen in temp_gen]
        enc_sal = [encrypt(sal, pk) for sal in temp_sal]
        server.submit_salary(enc_sal, enc_gen)
 
class GenderPayGapSurveyServer:
    def __init__(self):
        self.salaries = []
        self.genders = []
        self.sk, self.pk = keygen(32)
    
    def get_public_key(self):
        return self.pk
        
    def submit_salary(self, ct_salary_vector, ct_gender_vector):
        """Store an entry in the survey"""
        self.salaries.append(ct_salary_vector)
        self.genders.append(ct_gender_vector)
    
    def show_salaries(self):
        """Display the (encrypted) submitted salaries"""
        return self.salaries
    
    def compute_average_salaries(self):
        """Tally the results, decrypt, and return a 2-tuple: (average female salary, average male salary)"""
        full_dec_list = []
        for enc_vals in [self.salaries, self.genders]:
            arr_encs = np.array(enc_vals)
            col_sums = []
            for col in range(len(arr_encs[1])):
                sums = []
                sums.append(e_add(arr_encs[:, col][0], arr_encs[:, col][1], self.pk))
                for i in range(2, len(arr_encs[:, col])):
                    sums.append(e_add(sums[-1], arr_encs[:, col][i], self.pk))
                col_sums.append(sums[-1])
            dec_sums = [decrypt(sum, self.sk, self.pk) for sum in col_sums]
            full_dec_list.append(dec_sums)
        full_dec_list = np.array(full_dec_list)
        return (float(full_dec_list[:, 0][0]/full_dec_list[:, 0][1]) , float(full_dec_list[:, 1][0]/full_dec_list[:, 0][1])) 

In [4]:
s = GenderPayGapSurveyServer()
GenderPayGapSurveyParticipant().submit_salary(10000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(30000, 'Female', s)
GenderPayGapSurveyParticipant().submit_salary(15000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(20000, 'Female', s)
assert s.compute_average_salaries() == (25000.0, 12500.0)